<a href="https://colab.research.google.com/github/franciscogarate/cdiae/blob/main/notebooks/13_Mejor_Estimacion_Decesos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cálculo de la Mejor Estimación de una póliza de Decesos (Vida Entera)

En este ejemplo, calcularemos la mejor estimación de un de decsos (seguro vida) a prima única (sin entradas de primas).
El primer paso será descargarnos la base de datos clonando el repositorio de github:

In [99]:
!git clone https://github.com/franciscogarate/cdiae/

Importamos las librerias necesarias, así como la tabla de mortalidad y curva libre de riesgo

In [100]:
import pandas as pd
import numpy as np
from cdiae import lx, clr

Establezco las **hipótesis** de mi proyección:

In [101]:
lx = lx.PASEM2020_Decesos_M_2orden.values.copy() # Tabla PASEM2020 Decesos 2orden Hombres
edad = 50                                        # Edad en t=0
w = len(lx)                                      # Fin de tabla
capital = 5000.                                  # Capital inicial
tasa_incr_capital = 0.015                        # Incremento del capital
tasa_caida = 0.03                                # Caida de cartera
tasa_gastos = 0.05                               # Gastos por poliza sobre el capital

Creo un **df** con **pd.date_range()** para realizar mis proyecciones. Para simplificar, calculamos con flujos anuales.

In [102]:
df = pd.DataFrame(pd.date_range(start='2024-12-31',periods=(w - edad),freq='YE'), columns=['Fecha'])
df['edad'] = edad + df.index
df['t'] = df.index
df['lx'] = df['edad'].apply(lambda x: lx[x])
df.head()

,Fecha,edad,t,lx
0,2024-12-31,50,0,96712.87427
1,2025-12-31,51,1,96371.78194
2,2026-12-31,52,2,95995.34313
3,2027-12-31,53,3,95581.66597
4,2028-12-31,54,4,95128.98482


Calculo la qx como diferencia entre lx entre el número de supervivientes al inicio de la proyección:

In [103]:
df['qx'] = df['lx'].diff(-1).fillna(0)/df['lx'][0]
df['sum_qx'] = df['qx'].cumsum()
df['px'] = 1 - df['sum_qx']
df.head()

,Fecha,edad,t,lx,qx,sum_qx,px
0,2024-12-31,50,0,96712.87427,0.003527,0.003527,0.996473
1,2025-12-31,51,1,96371.78194,0.003892,0.007419,0.992581
2,2026-12-31,52,2,95995.34313,0.004277,0.011697,0.988303
3,2027-12-31,53,3,95581.66597,0.004681,0.016377,0.983623
4,2028-12-31,54,4,95128.98482,0.005103,0.021480,0.978520


Aplicamos nuestro incremento de capital

In [104]:
def incr_capital(capital, t):
  return capital * (1 + tasa_incr_capital) ** t

df['capital'] = incr_capital(capital, df.t)
df.head()

,Fecha,edad,t,lx,qx,sum_qx,px,capital
0,2024-12-31,50,0,96712.87427,0.003527,0.003527,0.996473,5000.000000
1,2025-12-31,51,1,96371.78194,0.003892,0.007419,0.992581,5075.000000
2,2026-12-31,52,2,95995.34313,0.004277,0.011697,0.988303,5151.125000
3,2027-12-31,53,3,95581.66597,0.004681,0.016377,0.983623,5228.391875
4,2028-12-31,54,4,95128.98482,0.005103,0.021480,0.978520,5306.817753


Calculamos los pagos probabilizados en casa periodo de la proyección. Sin incremento del capital, la suma de todos los períodos debería ser igual al capital inicial:

In [105]:
df['pagos'] = df['capital'] * df['qx']
df.head()

,Fecha,edad,t,lx,qx,sum_qx,px,capital,pagos
0,2024-12-31,50,0,96712.87427,0.003527,0.003527,0.996473,5000.000000,17.634277
1,2025-12-31,51,1,96371.78194,0.003892,0.007419,0.992581,5075.000000,19.753595
2,2026-12-31,52,2,95995.34313,0.004277,0.011697,0.988303,5151.125000,22.033290
3,2027-12-31,53,3,95581.66597,0.004681,0.016377,0.983623,5228.391875,24.472382
4,2028-12-31,54,4,95128.98482,0.005103,0.021480,0.978520,5306.817753,27.080739


In [106]:
df.pagos.sum()

np.float64(7975.318619403105)

Aplicamos nuestra tasa de caída de la cartera:

In [107]:
df['tasa_caida'] = tasa_caida
df['polizas'] = (1 - df['tasa_caida']).cumprod()
df.head()

,Fecha,edad,t,lx,qx,sum_qx,px,capital,pagos,tasa_caida,polizas
0,2024-12-31,50,0,96712.87427,0.003527,0.003527,0.996473,5000.000000,17.634277,0.03,0.970000
1,2025-12-31,51,1,96371.78194,0.003892,0.007419,0.992581,5075.000000,19.753595,0.03,0.940900
2,2026-12-31,52,2,95995.34313,0.004277,0.011697,0.988303,5151.125000,22.033290,0.03,0.912673
3,2027-12-31,53,3,95581.66597,0.004681,0.016377,0.983623,5228.391875,24.472382,0.03,0.885293
4,2028-12-31,54,4,95128.98482,0.005103,0.021480,0.978520,5306.817753,27.080739,0.03,0.858734


Calculamos cuantos beneficiarios probabilizados por fallecimiento y caída quedan al final de cada período:

In [108]:
df['polizas_benef'] =  df['px'] * df['polizas']
df.head(2)

,Fecha,edad,t,lx,qx,sum_qx,px,capital,pagos,tasa_caida,polizas,polizas_benef
0,2024-12-31,50,0,96712.87427,0.003527,0.003527,0.996473,5000.0,17.634277,0.03,0.9700,0.966579
1,2025-12-31,51,1,96371.78194,0.003892,0.007419,0.992581,5075.0,19.753595,0.03,0.9409,0.933919


En este paso, calculamos los pagos por siniestralidad probabilizados

In [109]:
df['pagos_prob'] = df['pagos'] * df['polizas_benef']
df.head(2)

,Fecha,edad,t,lx,qx,sum_qx,px,capital,pagos,tasa_caida,polizas,polizas_benef,pagos_prob
0,2024-12-31,50,0,96712.87427,0.003527,0.003527,0.996473,5000.0,17.634277,0.03,0.9700,0.966579,17.044921
1,2025-12-31,51,1,96371.78194,0.003892,0.007419,0.992581,5075.0,19.753595,0.03,0.9409,0.933919,18.448263


In [110]:
df['clr'] = df['t'].apply(lambda t: clr[t])
df['factor_desc'] = df.apply(lambda x: 1 / (1 + x.clr) ** (x.t), axis=1)
df.head(2)

,Fecha,edad,t,lx,qx,sum_qx,px,capital,pagos,tasa_caida,polizas,polizas_benef,pagos_prob,clr,factor_desc
0,2024-12-31,50,0,96712.87427,0.003527,0.003527,0.996473,5000.0,17.634277,0.03,0.9700,0.966579,17.044921,0.02236,1.000000
1,2025-12-31,51,1,96371.78194,0.003892,0.007419,0.992581,5075.0,19.753595,0.03,0.9409,0.933919,18.448263,0.02093,0.979499


Incluimos los gastos

In [111]:
df['gastos'] = df['capital'] * tasa_gastos
df['gastos_prob'] = df['gastos'] * df['polizas_benef']
df.head(2)

,Fecha,edad,t,lx,qx,sum_qx,px,capital,pagos,tasa_caida,polizas,polizas_benef,pagos_prob,clr,factor_desc,gastos,gastos_prob
0,2024-12-31,50,0,96712.87427,0.003527,0.003527,0.996473,5000.0,17.634277,0.03,0.9700,0.966579,17.044921,0.02236,1.000000,250.00,241.644738
1,2025-12-31,51,1,96371.78194,0.003892,0.007419,0.992581,5075.0,19.753595,0.03,0.9409,0.933919,18.448263,0.02093,0.979499,253.75,236.982018


Incluimos la curva libre de riesgo para calcular el factor de descuento:

In [112]:
df['salidas_prob'] = df['pagos_prob'] + df['gastos_prob']
df.head(2)

,Fecha,edad,t,lx,qx,sum_qx,px,capital,pagos,tasa_caida,polizas,polizas_benef,pagos_prob,clr,factor_desc,gastos,gastos_prob,salidas_prob
0,2024-12-31,50,0,96712.87427,0.003527,0.003527,0.996473,5000.0,17.634277,0.03,0.9700,0.966579,17.044921,0.02236,1.000000,250.00,241.644738,258.689659
1,2025-12-31,51,1,96371.78194,0.003892,0.007419,0.992581,5075.0,19.753595,0.03,0.9409,0.933919,18.448263,0.02093,0.979499,253.75,236.982018,255.430282


Por último, descontamos los flujos probabilizados utilizando el factor de descuento:

In [113]:
print(f'BEL: {(df.salidas_prob @ df.factor_desc):,.2f}')

BEL: 5,236.34
